# **ECE420 - Assignment 2**
### Saud Badar - 1002554595
### Kyu Bum Kim - 1003969100

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/ECE421/Lab2')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Load the data
def loadData():
    with np.load("notMNIST.npz") as data:
        Data, Target = data["images"], data["labels"]
        np.random.seed(521)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data = Data[randIndx] / 255.0
        Target = Target[randIndx]
        trainData, trainTarget = Data[:10000], Target[:10000]
        validData, validTarget = Data[10000:16000], Target[10000:16000]
        testData, testTarget = Data[16000:], Target[16000:]
    return trainData, validData, testData, trainTarget, validTarget, testTarget

In [0]:
def convertOneHot(trainTarget, validTarget, testTarget):
    newtrain = np.zeros((trainTarget.shape[0], 10))
    newvalid = np.zeros((validTarget.shape[0], 10))
    newtest = np.zeros((testTarget.shape[0], 10))

    for item in range(0, trainTarget.shape[0]):
        newtrain[item][trainTarget[item]] = 1
    for item in range(0, validTarget.shape[0]):
        newvalid[item][validTarget[item]] = 1
    for item in range(0, testTarget.shape[0]):
        newtest[item][testTarget[item]] = 1
    return newtrain, newvalid, newtest

In [0]:
def shuffle(trainData, trainTarget):
    np.random.seed(421)
    randIndx = np.arange(len(trainData))
    target = trainTarget
    np.random.shuffle(randIndx)
    data, target = trainData[randIndx], target[randIndx]
    return data, target

In [0]:
tf.reset_default_graph()
x = tf.placeholder("float", [None, 28,28,1])
true = tf.placeholder("float", [None, 10])

# This is the first pair of weights and biases
with tf.variable_scope("Weight_11", reuse = tf.AUTO_REUSE):
  weight_in = tf.get_variable('Weight_11', shape = (3, 3, 1, 32), initializer = tf.contrib.layers.xavier_initializer())
with tf.variable_scope("Bias_11", reuse = tf.AUTO_REUSE):
  bias_in =  tf.get_variable('Bias_11', shape = (32), initializer = tf.contrib.layers.xavier_initializer())

#This is the second pair of weights and biases for the weights between conv2d and fc1
with tf.variable_scope("Weight_22", reuse = tf.AUTO_REUSE):
  weight_fc1 = tf.get_variable('Weight_22', shape = (14*14*32, 784), initializer = tf.contrib.layers.xavier_initializer())
with tf.variable_scope("Bias_22", reuse = tf.AUTO_REUSE):
  bias_fc1 = tf.get_variable('Bias_22', shape = (784), initializer = tf.contrib.layers.xavier_initializer())

#This is the third pair of weights and biases for the weights between fc1 and fc2
with tf.variable_scope("Weight_33", reuse = tf.AUTO_REUSE):
  weight_fc2 = tf.get_variable('Weight_33', shape = (784, 10), initializer = tf.contrib.layers.xavier_initializer())
with tf.variable_scope("Bias_33", reuse = tf.AUTO_REUSE):
  bias_fc2 = tf.get_variable('Bias_33', shape = (10), initializer = tf.contrib.layers.xavier_initializer())

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
def conv2d(x, W, b):
    x = tf.nn.conv2d(x, W, strides =[1, 1, 1, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return (x)

def maxpool2d(x, size):
    x = tf.nn.max_pool(x, ksize=[1, size, size, 1], strides=[1, size, size, 1], padding='SAME')
    return(x)

trainData, validData, testData, trainTarget, validTarget, testTarget = loadData()
trainData = trainData.reshape(-1, 28, 28, 1)
validData = validData.reshape(-1, 28, 28, 1)
testData = testData.reshape(-1, 28, 28, 1)
trainTarget, validTarget, testTarget = convertOneHot(trainTarget, validTarget, testTarget)

In [0]:
learning_rate = 1e-4
batch_size = 32
epochs = 50
reg = 0.5
drop = 0.5

def CNN(x):
    #1. Input Layer(This has already been created in the code above)

    #2.A 3 × 3 convolutional layer, with 32 filters, using vertical and horizontal strides of 1.
    #(use my own fuction)
    convolutional_layer =  conv2d(x, weight_in, bias_in)

    #3. ReLU activation
    relu_layer = tf.nn.relu(convolutional_layer)

    #4. A batch normalization layer
    #4.1 Get the mean and varience
    batchMean, batchVar = tf.nn.moments(relu_layer, [0])
    #4.2 Normalizing the data
    normal_layer = tf.nn.batch_normalization(relu_layer, mean = batchMean, variance= batchVar, offset = 0, scale = None, variance_epsilon= 1e-8)

    #5. A 2 × 2 max pooling layer (use my own fuction)
    max_pool_layer = maxpool2d(normal_layer, 2)

    #6. Flatten layer
    flatten_layer = tf.reshape(max_pool_layer, [-1, weight_fc1.get_shape().as_list()[0]])

    #7. Fully connected layer (with 784 output units, i.e. corresponding to each pixel)
    fully_connected_1_unbiased = tf.matmul(flatten_layer, weight_fc1)
    fully_connected_one =  tf.add(fully_connected_1_unbiased, bias_fc1)

    #8. ReLU activation
    relu_layer_two = tf.nn.relu(fully_connected_one)

    #8.1 The hidden dropout layer
    if drop != 0:
      relu_layer_two = tf.nn.dropout(relu_layer_two, rate = drop)

    #9. Fully connected layer (with 10 output units, i.e. corresponding to each class)
    fully_connected_2_unbiased = tf.matmul(relu_layer_two, weight_fc2)
    out =  tf.add(fully_connected_2_unbiased, bias_fc2)

    return (out)

# In this step we get the value of 
output = CNN(x)

#10. Softmax output (For accuracy)
softmax = tf.nn.softmax(output)

#11. Cross Entropy loss
cost_CE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels= true, logits = output))

# regularizers = tf.nn.l2_loss(weight_in) + tf.nn.l2_loss(weight_fc1) + tf.nn.l2_loss(weight_fc2)
# cost_CE = tf.reduce_mean(cost_CE + reg * regularizers)

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost_CE)

#Prediction Calculation
compared_prediction = tf.equal(tf.argmax(softmax, 1), tf.argmax(true, 1))

#Accuracy Calculation
final_acc = tf.reduce_mean(tf.cast(compared_prediction, tf.float64))

In [0]:
train_loss, val_loss, test_loss = [], [], []
train_acc, val_acc, test_acc = [], [], []


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(epochs):
        batch_loop = len(trainData)//batch_size
        #Second for loop for the batch size
        for j in range(batch_loop):
            batch_start = j * batch_size
            batch_end = min(((j + 1) * batch_size), len(trainData))

            batch_data = trainData[batch_start:batch_end]
            
            batch_end = min(((j + 1) * batch_size), len(trainTarget))
            batch_target = trainTarget[batch_start: batch_end]
            optimize = sess.run(optimizer, feed_dict= {x: batch_data, true: batch_target})
            
        #Now we will calculate the loss for each iteration
        epoch_train_loss = sess.run(cost_CE, feed_dict={x: trainData, true: trainTarget})
        epoch_train_acc = sess.run(final_acc, feed_dict ={x: trainData, true: trainTarget})
        epoch_val_loss = sess.run(cost_CE, feed_dict={x: validData, true: validTarget})
        epoch_val_acc = sess.run(final_acc, feed_dict ={x: validData, true: validTarget})
        epoch_test_loss = sess.run(cost_CE, feed_dict={x: testData, true: testTarget})
        epoch_test_acc = sess.run(final_acc, feed_dict ={x: testData, true: testTarget})
        print("Number of epoch: {}".format(i))

        #Now we add it to the list
        train_loss.append(epoch_train_loss)
        train_acc.append(epoch_train_acc)
        val_loss.append(epoch_val_loss)
        val_acc.append(epoch_val_acc)
        test_loss.append(epoch_test_loss)
        test_acc.append(epoch_test_acc)

        #Now shuffle the data after each epoch
        trainData, trainTarget = shuffle(trainData, trainTarget)

#First Graph for the Loss
n = len(train_loss)
plt.title("Train, Validation and Test Loss (dropout = 0.75)")
n = len(train_loss) # number of epochs
plt.plot(range(1,n+1), train_loss, label="Train")
plt.plot(range(1,n+1), val_loss, label="Validation")
plt.plot(range(1, n+1), test_loss, label = "Test")
plt.xlabel("Epoch")
plt.ylabel("Cross Entropy Loss")
plt.legend(loc='best')
plt.show()

#Second Graph for the Accuracy
plt.title("Train, Validation and Test Accuracy (dropout = 0.75)")
plt.plot(range(1,n+1), train_acc, label="Train")
plt.plot(range(1,n+1), val_acc, label="Validation")
plt.plot(range(1, n+1), test_acc, label = "Test")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc='best')
plt.show()

n = len(train_loss)
print("This is the final Training Loss: {0:.4f}".format(train_loss[n - 1]))
print("This is the final Validation Loss: {0:.4f}".format(val_loss[n - 1]))
print("This is the final Testing Loss: {0:.4f}".format(test_loss[n - 1]))
print("This is the final Training Accuracy: {0:.4f}%".format(train_acc[n - 1] * 100))
print("This is the final Validation Accuracy: {0:.4f}%".format(val_acc[n - 1] * 100))
print("This is the final Testing Accuracy: {0:.4f}%".format(test_acc[n - 1] * 100))

#**THIS IS THE CODE FOR PART ONE**

In [0]:
def computeLayer(X, W, b):
    '''
    x (28*28, B)
    w_h(28*28, N)
    b_h (1, N)
    #(N, 28*28) * (28*28, B) + (N,1) -> (N, B)

    z (N, B)
    w_o(N, 10)
    b_o (1, 10)
    (10, N) * (N, B) + (10,1) -> (10, B)
    '''
    return ((X@W) +b)

def softmax(x):
  expos = np.exp(x - np.max(x))
  return expos / np.sum(expos, axis=1).reshape(-1, 1)

def relu(x):
    return np.maximum(0, x)

def gradCE(target, prediction):
    return softmax(prediction) - target

def gradRelu(x):
    x[x<0] = 0
    x[x>0] = 1
    return x

def CE(target, prediction):
  return -1*(target*np.log(prediction)).mean()

def accuracy(prediction, target): 
  #(b, 10) (b, 10)
    # print(prediction)
    max_indices_pred = prediction.argmax(axis=1)
    indices_target = target.argmax(axis=1)
    # print(max_indices_pred, indices_target)
    accurate_arr = np.equal(max_indices_pred, indices_target)
    return np.sum((accurate_arr==True))/(indices_target.shape[0])

In [0]:
def forward_propagation(x, W_h, b_h, W_o, b_o):
    s_h = computeLayer(x, W_h, b_h)
    x_h = relu(s_h)
    s_o = computeLayer(x_h, W_o, b_o)
    x_o = softmax(s_o)
    return (s_h, x_h, s_o, x_o)

def back_propagation(trainTarget, trainData, s_h, s_o, x_h, W_o):
    change_out = gradCE(trainTarget, s_o)
    grad_relu = gradRelu(s_h)
    step = np.matmul(change_out,W_o.transpose())
    change_hidden = grad_relu * step

    grad_W_h = np.matmul(trainData.transpose(), change_hidden)
    grad_W_o = np.matmul(x_h.transpose(), change_out)
    grad_b_h = np.sum(change_hidden, axis = 0)
    grad_b_o = np.sum(change_out, axis = 0)

    return(grad_W_h, grad_W_o, grad_b_h, grad_b_o)

In [0]:
def NN():
    np.random.seed(21)
    trainData, validData, testData, trainTarget, validTarget, testTarget = loadData()
    trainTarget, validTarget, testTarget = convertOneHot(trainTarget, validTarget, testTarget)

    #Change the shape of all the dataset
    trainData = trainData.reshape((trainData.shape[0], trainData.shape[1]*trainData.shape[2]))
    validData = validData.reshape((validData.shape[0], validData.shape[1]*validData.shape[2]))
    testData = testData.reshape((testData.shape[0], testData.shape[1]*testData.shape[2]))

    #Set up the parameters
    hidden_units = 2000
    num_epochs = 200
    classes = 10
    learning_rate = 1e-5
    gamma = 0.99

    #Set up the array for the loss and accuracy
    train_loss, val_loss, test_loss = [], [], []
    train_acc, val_acc, test_acc = [], [], []


    #Set up all the weights and bias following Xiaver initiazation scheme
    W_h = np.random.normal(0, np.sqrt(2.0/(784 + hidden_units)), (784, hidden_units))
    W_o = np.random.normal(0, np.sqrt(2.0/(classes + hidden_units)), (hidden_units, classes))

    b_h = np.zeros((1, hidden_units))
    b_o = np.zeros((1, classes))

    #Set up all the learning weights
    vW_h = np.full((784, hidden_units), 1e-5)
    vW_o = np.full((hidden_units, classes), 1e-5)
    vb_h = np.full((1, hidden_units), 1e-5)
    vb_o = np.full((1, classes), 1e-5)

    for epoch in range(num_epochs):
          #Shuffle the data each epoch
          trainData, trainTarget = shuffle(trainData, trainTarget)

          #Forward propagation
          s_h, x_h, s_o, x_o = forward_propagation(trainData, W_h, b_h, W_o, b_o)

          #Back propagation
          grad_W_h, grad_W_o, grad_b_h, grad_b_o = back_propagation(trainTarget, trainData, s_h, s_o, x_h, W_o)

          #Change the learning weights
          vW_h = (learning_rate*grad_W_h) + (gamma*vW_h)
          vW_o = (learning_rate*grad_W_o) + (gamma*vW_o)
          vb_h = (learning_rate*grad_b_h) + (gamma*vb_h)
          vb_o = (learning_rate*grad_b_o) + (gamma*vb_o)

          #Add the learning weight to the actual weights
          W_h = W_h - vW_h
          W_o = W_o - vW_o
          b_h = b_h - vb_h
          b_o = b_o - vb_o

          #Find the accuracy for the training dataset
          acc = accuracy(x_o, trainTarget)
          train_acc.append(acc*100)

          #Find the loss for the training dataset
          loss = CE(trainTarget, x_o)
          train_loss.append(loss)

          #Find the x_o for the validation set
          s_h_val, x_h_val, s_o_val, x_o_val = forward_propagation(validData, W_h, b_h, W_o, b_o)

          #Find the accuracy for the validation dataset
          acc = accuracy(x_o_val, validTarget)
          val_acc.append(acc*100)

          #Find the loss for the validation dataset
          loss = CE(validTarget, x_o_val)
          val_loss.append(loss)

          #Find the x_o for the test set
          s_h_test, x_h_test, s_o_test, x_o_test = forward_propagation(testData, W_h, b_h, W_o, b_o)

          #Find the accuracy for the testing dataset)
          acc = accuracy(x_o_test, testTarget)
          test_acc.append(acc*100)

          #Find the loss for the testing dataset
          loss = CE(testTarget, x_o_test)
          test_loss.append(loss)  
          print("Epoch: {}".format(epoch))
    
    #First Graph for the Loss
    n = len(train_loss)
    plt.title("Train, Validation and Test Loss ")
    plt.plot(range(1,n+1), train_loss, label="Train")
    plt.plot(range(1,n+1), val_loss, label="Validation")
    plt.plot(range(1, n+1), test_loss, label = "Test")
    plt.xlabel("Epochs")
    plt.ylabel("Cross Entropy Loss")
    plt.legend(loc='best')
    plt.show()

    #Second Graph for the Accuracy
    n = len(train_acc)
    plt.title("Train, Validation and Test Accuracy")
    plt.plot(range(1,n+1), train_acc, label="Train")
    plt.plot(range(1,n+1), val_acc, label="Validation")
    plt.plot(range(1, n+1), test_acc, label = "Test")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

    n = len(train_loss)
    print("This is the final Training Loss: {0:.4f}".format(train_loss[n - 1]))
    print("This is the final Validation Loss: {0:.4f}".format(val_loss[n - 1]))
    print("This is the final Testing Loss: {0:.4f}".format(test_loss[n - 1]))
    print("This is the final Training Accuracy: {0:.4f}%".format(train_acc[n - 1]))
    print("This is the final Validation Accuracy: {0:.4f}%".format(val_acc[n - 1]))
    print("This is the final Testing Accuracy: {0:.4f}%".format(test_acc[n - 1]))

    return(n)

x = NN()

#**PART ONE VERSION 2**

In [0]:
# def train(train_data, valid_data, test_data, train_target, valid_target, test_target, epochs, alpha, gamma, neurons):
#     '''
#     train data (B, 28*28)
#     train_target (B, 10)
#     '''

#     stddev_h = np.sqrt(2/(train_data.shape[0] + neurons))
#     w_h = np.random.normal(0, stddev_h, (train_data.shape[1], neurons)) #(28*28, N)
#     v_h = np.full((train_data.shape[1], neurons), 10**-5) #(28*28, N)
#     b_h = np.zeros((1, neurons)) #(1, N)

#     stddev_o = np.sqrt(2/(neurons+10))
#     w_o = np.random.normal(0, stddev_o, (neurons,10)) #(N, 10)
#     v_o = np.full((neurons, 10), 10**-5)  #(N, 10)
#     b_o = np.zeros((1, 10)) #(1, 10)
   

#     train_loss, val_loss, test_loss, train_acc, val_acc, test_acc = [], [], [], [], [], []

#     for epoch in range(epochs):
#         print('Epoch: ' + str(epoch))

#         #test test
#         y, s2, s1, z = for_prop(test_data, w_o, w_h, b_o, b_h)
#         loss = CE(test_target, y)
#         test_loss.append(loss)
#         acc = accuracy(y, test_target)
#         test_acc.append(acc)
        
#         #test valid
#         y, s2, s1, z = for_prop(valid_data, w_o, w_h, b_o, b_h)
#         loss = CE(valid_target, y)
#         val_loss.append(loss)
#         acc = accuracy(y, valid_target)
#         val_acc.append(acc)

#         #test train
#         y, s2, s1, z = for_prop(train_data, w_o, w_h, b_o, b_h)
#         loss = CE(train_target, y)
#         train_loss.append(loss)
#         acc = accuracy(y, train_target)
#         train_acc.append(acc)
        
#         #back prop
#         dldw_o, dldb_o, dldw_h, dldb_h = back_prop(train_target, train_data, w_o, z, s1, y)
        
#         #Update the gradients 
#         b_h -= alpha * dldb_h
#         b_o -= alpha * dldb_o

#         v_h = gamma * v_h + alpha * dldw_h
#         v_o = gamma * v_o + alpha * dldw_o

#         w_h -= v_h 
#         w_o -= v_o 

      

#     plot(epochs, train_loss, val_loss, test_loss, train_acc, val_acc, test_acc)
    
# def plot(epochs, train_loss, val_loss, test_loss, train_acc, val_acc, test_acc):
#     plt.figure(figsize=(20,10))
#     plt.title("Training Curve Loss")
#     plt.plot(range(epochs), train_loss, label="Train loss")
#     plt.plot(range(epochs), val_loss, label="Valid loss")
#     plt.plot(range(epochs), test_loss, label="Test loss")
#     plt.xlabel("Epochs")
#     plt.ylabel("Loss")
#     plt.legend()
#     plt.show()

#     plt.figure(figsize=(20,10))
#     plt.title("Training Curve Accuracy")
#     plt.plot(range(epochs), train_acc, label="Train accuracy")
#     plt.plot(range(epochs), val_acc, label="Validation accuracy")
#     plt.plot(range(epochs), test_acc, label="Test accuracy")
#     plt.xlabel("Epochs")
#     plt.ylabel("Training Accuracy")
#     plt.legend()
#     plt.show()
    
#     print('Most Recent Training Accuracy = ' + str(train_acc[epochs-1]))
#     print('Most Recent Validation Accuracy = ' + str(val_acc[epochs-1]))
#     print('Most Recent Test Accuracy = ' + str(test_acc[epochs-1]))
#     print('Max Training Accuracy = ' + str(max(train_acc)))
#     print('Max Validation Accuracy = ' + str(max(val_acc)))
#     print('Max Test Accuracy = ' + str(max(test_acc)))
#     print('Most Recent Training Loss = ' + str(train_loss[epochs-1]))
#     print('Most Recent Validation Loss = ' + str(val_loss[epochs-1]))
#     print('Most Recent Test Loss = ' + str(test_loss[epochs-1]))
#     print('Min Training Loss = ' + str(min(train_loss)))
#     print('Min Validation Loss = ' + str(min(val_loss)))
#     print('Min Test Loss = ' + str(min(test_loss)))